In [1]:
from goes2go import GOES
import pandas as pd
from datetime import datetime
import xarray as xr
import subprocess
from netCDF4 import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np

### GOES-16 Satellite and Cloud Products

The **GOES-16** satellite, part of the NOAA Geostationary Operational Environmental Satellites (GOES) series, provides advanced imaging capabilities for monitoring weather and environmental phenomena. GOES-16 operates in geostationary orbit, continuously observing the same area of the Earth, and it is particularly useful for weather forecasting, environmental monitoring, and disaster response. It collects a range of products related to clouds, moisture, solar radiation, and more, which are available through platforms like `GOES2GO`.

**GOES2GO** is a Python package that allows easy access to GOES satellite data, facilitating the download and processing of various products from the satellite's instruments.

Below are the products you are working with:

- **ABI-L2-MCMIPC (Multi-Channel Cloud and Moisture Imagery Product)**: This product provides imagery from multiple channels of the Advanced Baseline Imager (ABI) instrument, delivering detailed information about clouds and moisture in the atmosphere. It is useful for visualizing cloud structures, monitoring moisture patterns, and aiding in severe weather detection.

- **ABI-L2-CODC (Cloud Optical Depth Product)**: The Cloud Optical Depth product provides information about the thickness and density of clouds by measuring how much sunlight is transmitted through them. This product helps in understanding cloud properties and their impact on the Earth's radiation balance.

- **ABI-L2-RSRC (Reflected Shortwave Radiation Product)**: This product measures the amount of shortwave solar radiation reflected by the Earth's surface and atmosphere. It is important for assessing the Earth's energy budget and understanding how clouds and aerosols affect the reflection of solar energy.

Each of these products plays a key role in monitoring and analyzing weather patterns, atmospheric conditions, and energy exchanges in the Earth's atmosphere.


In [2]:
cloud_and_moisture_download = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')

cloud_optical_depth_download = GOES(satellite=16, product="ABI-L2-CODC", domain='C')

reflected_sw_downlad = GOES(satellite=16, product="ABI-L2-RSRC", domain='C')

### Downloading GOES-16 Cloud and Radiation Products for a Specific Time Period

In this step, we use the `GOES2GO` package to download data from **GOES-16** for the time period between April 1, 2020, 18:00 UTC, and April 11, 2020, 20:00 UTC. We are downloading data for three specific products:

- **Multi-Channel Cloud and Moisture Product (ABI-L2-MCMIPC)**: Provides cloud and moisture imagery.
- **Cloud Optical Depth Product (ABI-L2-CODC)**: Gives information about the optical thickness of clouds.
- **Reflected Shortwave Radiation Product (ABI-L2-RSRC)**: Measures shortwave solar radiation reflected by clouds and the Earth's surface.

The data for each product is downloaded as a pandas DataFrame using the `df()` method from the `GOES2GO` package. The start and end times for the download are specified to capture the desired time range.


In [3]:
df_cloud_and_moisture = cloud_and_moisture_download.df(start='2020-04-01 18:00', end='2020-04-11 20:00')
df_cloud_optical_depth = cloud_optical_depth_download.df(start='2020-04-01 18:00', end='2020-04-11 20:00')
df_reflected_sw = reflected_sw_downlad.df(start='2020-04-01 18:00', end='2020-04-11 20:00')

### Function to Download GOES-16 Files for Specified Julian Days

The `get_files16` function is designed to download specific GOES-16 files for a given time period, focusing on data nearest to 19:00 UTC (12 PM PDT) each day. The key steps involved are:

- **Julian Days**: We specify a list of Julian days (91 to 100), corresponding to April 1 to April 10.
- **Desired Time**: The function filters data to find the closest match to 19:00 UTC (the equivalent of 12 PM PDT) each day.
- **File Selection**: It finds the row with the minimum time difference to 19:00 UTC for each day.
- **File Download**: For each selected file, the corresponding file path is constructed and downloaded using the AWS CLI.
- **Output**: The function returns a list of local file paths where the files are downloaded.

The function works by:

1. **Processing the DataFrame**: It converts the `start` column to pandas datetime, calculates the time difference from 19:00 UTC for each row, and selects the rows with the smallest time difference per day.
2. **File Downloading**: It loops through the selected files and Julian days, constructs the S3 URLs and local file paths, and downloads the files using AWS CLI.
3. **Output**: The function returns a list of local file paths where the files are stored locally.


In [4]:
def get_files16(df, dataset_name):
    julian_day = [91, 92, 93, 94, 95, 96, 97, 98, 99, 100]  # Julian days for April 1 to April 10

    # Define the desired time in UTC for 12 PM PDT (which is 19:00 UTC)
    desired_time = pd.to_datetime("19:00:00")

    # Convert the datetime column of your dataframe to pandas datetime (assuming it's not already)
    df['datetime'] = pd.to_datetime(df['start'])

    # Create a new column with only the date part of the datetime column
    df['date'] = df['datetime'].dt.date

    # Calculate the time difference between each datetime and 19:00 UTC
    df['time_diff'] = (df['datetime'] - df['datetime'].dt.normalize() + pd.to_timedelta("19:00:00")).abs()

    # Group by date and find the row that has the smallest time difference
    nearest_19_utc = df.loc[df.groupby('date')['time_diff'].idxmin()]

    # Extract the files and nearest times
    nearest_19_utc_with_files = df.loc[nearest_19_utc.index, ['datetime', 'start', 'file']]

    # Convert the 'file' column to a Python list
    file_list = nearest_19_utc_with_files['file'].tolist()

    # Initialize a list to store local file paths
    local_file_paths = []

    # Loop through file_list and julian_day
    for file, day in zip(file_list, julian_day):
        # Correctly format the s3 file URL
        s3_file_url = f's3://{file}'

        # Correctly format the local file path with dataset_name and Julian day
        local_file_path = f'C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_{dataset_name}_2022day{day}.nc'
        
        # Add local file path to the list
        local_file_paths.append(local_file_path)

        # Use AWS CLI to download the file
        subprocess.run(['aws', 's3', 'cp', s3_file_url, local_file_path, '--no-sign-request'])

        print(f"File downloaded successfully to {local_file_path}")

    # Return both the list of file URLs and the local file paths
    return local_file_paths


### Downloading GOES-16 Files for Cloud and Radiation Products

In this step, we are calling the `get_files16` function to download the GOES-16 files for three specific products: **Cloud and Moisture Imagery**, **Cloud Optical Depth**, and **Reflected Shortwave Radiation**. The function will download files corresponding to the nearest time (19:00 UTC) for each day from April 1 to April 10, 2020. Each product is saved under a descriptive dataset name, and the file download is handled by AWS CLI.

1. **Cloud and Moisture Imagery**: 
   - The first `get_files16` call downloads files for the **Cloud and Moisture Product (ABI-L2-MCMIPC)**.
   
2. **Cloud Optical Depth**:
   - The second `get_files16` call downloads files for the **Cloud Optical Depth Product (ABI-L2-CODC)**.
   
3. **Reflected Shortwave Radiation**:
   - The third `get_files16` call downloads files for the **Reflected Shortwave Radiation Product (ABI-L2-RSRC)**.

The output includes a list of local file paths for each product.

In [5]:
print('Downloading files for Cloud and Moisture Product')
file_list_cloud_and_moisture = get_files16(df_cloud_and_moisture, 'CloudAndMoistureImagery')
print('Downloading files for Cloud Optical Depth')
file_list_cloud_optical_depth = get_files16(df_cloud_optical_depth, 'CloudOpticalDepth')
print('Downloading files for Reflected Shortwave Radiation')
file_list_reflected_sw = get_files16(df_reflected_sw, 'ReflectedSW')

File downloaded successfully to C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_CloudAndMoistureImagery_2022day91.nc
File downloaded successfully to C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_CloudAndMoistureImagery_2022day92.nc
File downloaded successfully to C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_CloudAndMoistureImagery_2022day93.nc
File downloaded successfully to C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_CloudAndMoistureImagery_2022day94.nc
File downloaded successfully to C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_CloudAndMoistureImagery_2022day95.nc
File downloaded successfully to C:/Users/adhal/OneDrive/Season3/Fall2024/MachineLearning/Data/GOESCloudMoistureImagery/GOES16_CloudAndMoistureImagery_2022day96.nc
File downloaded succes

In [6]:
print('Saving files for Cloud and Moisture Product')
np.save('file_list_cloud_and_moisture.npy', file_list_cloud_and_moisture)
print('Saving files for Cloud Optical Depth')
np.save('file_list_cloud_optical_depth.npy', file_list_cloud_optical_depth)
print('Saving files for Reflected Shortwave Radiation')
np.save('file_list_reflected_sw.npy', file_list_reflected_sw)

Saving files for Cloud and Moisture Product
Saving files for Cloud Optical Depth
Saving files for Reflected Shortwave Radiation
